In [ ]:
%load_ext autoreload
%autoreload 2
from Users.project.src.deep_learning_pipeline.context import Context
from Users.project.src.deep_learning_pipeline.loss_func import MSELossFuncCreator
from Users.project.src.deep_learning_pipeline.optimizer import AdamOptimizerCreator
from Users.project.src.base.logger import ConsoleLogger, FileLogger
from Users.project.src.deep_learning_pipeline.learning_input import LearningInput
from Users.project.src.deep_learning_pipeline.model_creator import ModelManager
from Users.project.src.deep_learning_pipeline.scaler_manager import ScalerManager
from Users.project.src.predict_lab_time_module.all_track_model import SimpleModel_128_Hidden_1_Creator, SimpleModel_128_Hidden_2_Creator
from Users.project.src.predict_lab_time_module.all_track_model import SimpleModel_64_Hidden_2_Creator, SimpleModel_64_Hidden_1_Creator
from sklearn.preprocessing import StandardScaler
import torch
import pandas as pd
import numpy as np
import os
from torch import nn, optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

data_save_dir = os.path.join("Users", "project", "data", "lap_time_predict")
features_path = os.path.join(data_save_dir, "features.csv")
features = pd.read_csv(features_path)
labels_path = os.path.join(data_save_dir, "labels.csv")
labels = pd.read_csv(labels_path)

model_path = os.path.join("Users", "project", "model", "lap_time_predict_test_4")
log_path = os.path.join("Users", "project", "log", "lap_time_predict_test_4")

model_creators = [SimpleModel_128_Hidden_1_Creator(), SimpleModel_128_Hidden_2_Creator()
                  , SimpleModel_64_Hidden_1_Creator(), SimpleModel_64_Hidden_2_Creator()]

optimizer_creator = AdamOptimizerCreator()
loss_func_creator = MSELossFuncCreator()

learning_input = LearningInput(features, labels)


cuda


In [2]:
%load_ext autoreload
%autoreload 2

logger = ConsoleLogger()
for i in range(10):
    model_save_path = os.path.join(model_path, f"step{i+1}")
    os.makedirs(model_save_path, exist_ok=True)
    for model_creator in model_creators:
        best_model_path = os.path.join(model_save_path, model_creator.get_save_name() + f"checkpoint.pth")
        scaler_save_path = os.path.join(model_save_path, model_creator.get_save_name() + f"scaler.joblib")
        scaler_manager = ScalerManager(scaler_save_path)
        model_manager = ModelManager(best_model_path)
        context = Context(device, model_manager, scaler_manager)
        context.train(learning_input, StandardScaler(), model_creator, optimizer_creator, loss_func_creator, logger
                          , 0.2, 42, 0.001, 200)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Epoch [10/200], Train Loss: 0.1809, Validation Loss: 0.1117
Epoch [20/200], Train Loss: 0.1199, Validation Loss: 0.0260
Epoch [30/200], Train Loss: 0.0739, Validation Loss: 0.0067
Epoch [40/200], Train Loss: 0.0541, Validation Loss: 0.3633
Epoch [50/200], Train Loss: 0.0371, Validation Loss: 0.0288
Epoch [60/200], Train Loss: 0.0794, Validation Loss: 0.0041
Epoch [70/200], Train Loss: 0.0375, Validation Loss: 0.0369
Epoch [80/200], Train Loss: 0.0357, Validation Loss: 0.0105
Epoch [90/200], Train Loss: 0.0298, Validation Loss: 0.0016
Epoch [100/200], Train Loss: 0.0227, Validation Loss: 0.0032
Epoch [110/200], Train Loss: 0.0229, Validation Loss: 0.0122
Epoch [120/200], Train Loss: 0.0184, Validation Loss: 0.0116
Epoch [130/200], Train Loss: 0.0250, Validation Loss: 0.0030
Epoch [140/200], Train Loss: 0.0152, Validation Loss: 0.0013
Epoch [150/200], Train Loss: 0.0159, Validation Loss: 0.0125
Epoch [160/200], Train Loss: 0.0191, Validation Loss: 0.0004
Epoch [170/200], Train Loss: 0.01

In [6]:
%load_ext autoreload
%autoreload 2
from Users.project.src.base.logger import ConsoleLogger

logger = FileLogger()
# model_creators = [SimpleModel_128_Hidden_1_Creator(), SimpleModel_128_Hidden_2_Creator()
#                   , SimpleModel_64_Hidden_1_Creator(), SimpleModel_64_Hidden_2_Creator()]

for i in range(10):
    model_save_path = os.path.join(model_path, f"step{i+1}")
    log_save_path = os.path.join(log_path, f"step{i+1}")
    os.makedirs(log_save_path, exist_ok=True)
    for model_creator in model_creators:
        best_model_path = os.path.join(model_save_path, model_creator.get_save_name() + f"checkpoint.pth")
        scaler_save_path = os.path.join(model_save_path, model_creator.get_save_name() + f"scaler.joblib")
        f_log_save_path = os.path.join(log_save_path, model_creator.get_save_name() + f"log.txt")
        scaler_manager = ScalerManager(scaler_save_path)
        model_manager = ModelManager(best_model_path)
        context = Context(device, model_manager, scaler_manager)
        logger.open(f_log_save_path, "w")
        context.test_df(features, labels, model_creator, logger)
        logger.close()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# from Users.project.src.data_container.data_container import AzureStorageAccess


# s = set()
# data_access = AzureStorageAccess()
# for file in data_access.get_all_file():
#     if "race_control_messages.csv" not in file.name:
#         continue
#     file_name = file.name
#     df = data_access.read_csv_from_blob(file_name)
#     for i in df["Message"]:
#         if "SAFETY" in i:
#             s.add(i)

# for i in s:
#     print(i)